In [1]:
import os

list_file = sorted(list(os.walk('./file/2.item'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['crawler_1.xlsx', 'crawler_2.xlsx']

In [2]:
file = list_file[0]

file

'crawler_1.xlsx'

In [3]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [4]:
input_ = pd.read_excel(f'./file/2.item/{file}',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Item Number'].tolist()

总数量：418


['235461387733',
 '233621258535',
 '334922305483',
 '255586148611',
 '234277033709',
 '232315264904',
 '235123315262',
 '256372876426',
 '235443864456',
 '256209081025',
 '256291287123',
 '335372891937',
 '235544392154',
 '335057592559',
 '232435277242',
 '332198105466',
 '235055850780',
 '232315242152',
 '255531675896',
 '232628030166',
 '235003975198',
 '234219682190',
 '335058269621',
 '255218432305',
 '235506777460',
 '256264372483',
 '334437920709',
 '233591979628',
 '335270739966',
 '256421824484',
 '235647995819',
 '255018389434',
 '335372988098',
 '256236108396',
 '256558512167',
 '234095593488',
 '234440531333',
 '334956110193',
 '256496980460',
 '334211431931',
 '332361055949',
 '235648003016',
 '255628697516',
 '335333813239',
 '256564411823',
 '235263199469',
 '256202118103',
 '235680200465',
 '234608488320',
 '256511996154',
 '256160146443',
 '255528777251',
 '255669513604',
 '235544303884',
 '334956170740',
 '335215802937',
 '256244140588',
 '256469386720',
 '334041548817

In [5]:
a = 0

a

0

In [6]:
crawler_status = 'error'

crawler_status

'error'

In [7]:
import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.get(input_.loc[a, 'Description Url'],
                            headers=get_header(ua=False),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200:
            break
    except:
        continue

resp

1


<Response [200]>

In [8]:
from bs4 import BeautifulSoup
from lxml import etree

soup = BeautifulSoup(resp.text, 'lxml')
html = etree.HTML(str(soup))

with open('./description.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <link href="//ir.ebaystatic.com" rel="dns-prefetch"/>
  <link href="//secureir.ebaystatic.com" rel="dns-prefetch"/>
  <link href="//i.ebayimg.com" rel="dns-prefetch"/>
  <link href="//rover.ebay.com" rel="dns-prefetch"/>
  <script>
   $ssgST=new Date().getTime();
  </script>
  <link as="font" fetchpriority="high" href="https://ir.ebaystatic.com/cr/v/c1/market-sans/v1.0/MarketSans-Regular-WebS.woff2" rel="preload" type="font/woff2"/>
  <style nonce="tMtwoRgE3LWeyp9YDhe78g==">
   body { font-family: "Market Sans", Arial, sans-serif; }
  </style>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <title>
   eBay
  </title>
  <link href="https://ir.ebaystatic.com/rs/c/item-desc-2Vk6qq44.css" rel="stylesheet"/>
 </head>
 <body>
  <script nonce="tMtwoRgE3LWeyp9YDhe78g==">
   (window.$ebay||($ebay = {})).tracking={pageId:'4487351',pageName:'videsc'}
  </script>
  <!--M#s0-14-->
  <div class="x-item-des

In [9]:
import re

list_text = html.xpath('//div[@class="x-item-description-child"]/center/table/tbody/tr[2]/td/table[2]/tbody/tr[2]/td[@class="text"]/div/descendant-or-self::strong')
list_text = [etree.tostring(text).decode('utf-8') for text in list_text]
list_text = [re.sub(r'(<br/>)+', '<br/>', text) for text in list_text]
list_text = [re.sub(r'<.*?>', ' ', text.replace('<br/>', '\n')) for text in list_text]
list_text = [re.sub(r' +', ' ', text).strip() for text in list_text]

list_text

[]

In [10]:
df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Item Number': input_.loc[a, 'Item Number'],
                         'Description Url': input_.loc[a, 'Description Url']}])

for i, text in zip(range(len(list_text)), list_text):
    df_temp.loc[0, f'Text {i+1}'] = text

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Item Number,Description Url
0,1,235461387733,https://vi.vipr.ebaydesc.com/itmdesc/235461387...


In [11]:
crawler_status = 'ok'

crawler_status

'ok'

In [12]:
if crawler_status == 'error':
    output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

output_error

""


In [13]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_description.xlsx', index=False)

output_correct

,No,Item Number,Description Url
0,1,235461387733,https://vi.vipr.ebaydesc.com/itmdesc/235461387...


In [14]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_description_error.xlsx', index=False)

output_error

""
